In [1]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [9]:
tokenizer=AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model=AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

C:\Users\Shaktimaan\anaconda3\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shaktimaan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [10]:
tokens= tokenizer.encode('I hated this, absolutely the worst', return_tensors='pt')

In [13]:
tokens[0]

tensor([  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 10103, 43060,
          102])

In [18]:
result=model(tokens)

In [19]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [72]:
r= requests.get('https://www.imdb.com/title/tt11737520/reviews?ref_=tt_urv')
soup= BeautifulSoup(r.text, 'html.parser')
regex= re.compile('.*text show-more__control.*')
results= soup.find_all('div',{'class':regex})
reviews=[result.text for result in results]

In [73]:
reviews[0]


"I have to admit that I've never watched any of the animated show One Piece or even heard about it for that matter. I'm really not a big fan of anime. Then after watching the trailers for this I thought it looked like a show for children. I was wrong. I kept reading about how good this show is and how it's #1 on Netflix so I thought I'd give it a chance. I'm glad I did because I couldn't have been more wrong. This is a show where people of all ages can enjoy. If you can just turn off your brain and watch I promise you'll have a good time. If you're looking for a new show to binge then give One Piece a try. You won't be disappointed!"

In [52]:
import pandas as pd
import numpy as np


In [77]:
df= pd.DataFrame(np.array(reviews), columns=['review'])

In [89]:
def sentiment_score(review):
    tokens= tokenizer.encode(review, return_tensors='pt')
    result= model(tokens)
    return int(torch.argmax(result.logits))+1

In [90]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [92]:
df

,review,sentiment
0,I have to admit that I've never watched any of...,4
1,I'm a 60yo man & old school like the The Marin...,5
2,Every live action anime that I've briefly seen...,5
3,This is bar none one of the best live-action a...,5
4,"Being a one piece fan myself, I was a bit inse...",5
5,One Piece has definitely lived up to all the h...,4
6,GREAT ADAPTATION! As an old-school fan (since ...,5
7,My kids play and watch One Piece anime and lov...,5
8,Initially I was apprehensive because of anime ...,5
9,Parenthetically I don't know anything about th...,4


In [93]:
df['review'].iloc[15]

"One Piece is one heck of an anime series that eludes anyone adapting to live action. The amount of special effects would be ridiculous. With the rubbery moves and silliness, it's an expensive challenge to make.Netflix' One Piece under the Japanese producers' watch eyes is a good adaptation. It might not be perfect, but from the storyline, casting and atory telling, it is One Piece.The directors took 1-2 steps backwards in term of CGI is a wise decision. Season 1 focused on the main crew recruitment was pretty much exactly what initial One Piece episodes are.Now that the main crew had been gathered, the series would face a much more difficult challenges due to the bigger crew recruitments and enemies.The casting was impressive, each Straw Hat Pirate member was matched accordingly that when you're an old fan of One Piece, you woukd recognize them in an instant. While if you're a new audience, you'll enjoy how tight the Straw Hat Pirate crew was illustrated.Good job, hopefully the 2nd se